In [56]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import patsy

import sklearn

from sklearn import svm
from sklearn import datasets



In [57]:
data_path = '898ce544-0-MLChallenge_1Datac63af4e/train_indessa.csv'

train_data = pd.read_csv(data_path)

train_data = train_data.fillna(0)


In [58]:
train_data.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,batch_enrolled,int_rate,grade,sub_grade,emp_title,...,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,verification_status_joint,last_week_pay,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,loan_status
0,58189336,14350,14350,14350.0,36 months,,19.19,E,E3,clerk,...,0.0,74.0,INDIVIDUAL,0,26th week,0.0,0.0,28699.0,30800.0,0
1,70011223,4800,4800,4800.0,36 months,BAT1586599,10.99,B,B4,Human Resources Specialist,...,0.0,0.0,INDIVIDUAL,0,9th week,0.0,0.0,9974.0,32900.0,0
2,70255675,10000,10000,10000.0,36 months,BAT1586599,7.26,A,A4,Driver,...,0.0,0.0,INDIVIDUAL,0,9th week,0.0,65.0,38295.0,34900.0,0
3,1893936,15000,15000,15000.0,36 months,BAT4808022,19.72,D,D5,Us office of Personnel Management,...,0.0,0.0,INDIVIDUAL,0,135th week,0.0,0.0,55564.0,24700.0,0
4,7652106,16000,16000,16000.0,36 months,BAT2833642,10.64,B,B2,LAUSD-HOLLYWOOD HIGH SCHOOL,...,0.0,0.0,INDIVIDUAL,0,96th week,0.0,0.0,47159.0,47033.0,0


In [59]:
test_data_path = '898ce544-0-MLChallenge_1Datac63af4e/test_indessa.csv'

test_data = pd.read_csv(test_data_path)

test_data['loan_status'] = (train_data.loan_status > 0).astype(int)

In [60]:
test_data = test_data.fillna(0)

In [61]:
test_data.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,batch_enrolled,int_rate,grade,sub_grade,emp_title,...,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,verification_status_joint,last_week_pay,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,loan_status
0,11937648,14000,14000,14000.0,60 months,BAT4711174,16.24,C,C5,Data Analyst,...,0.0,0.0,INDIVIDUAL,0,104th week,0.0,0.0,85230.0,45700.0,0
1,38983318,16000,16000,16000.0,60 months,BAT4318899,9.49,B,B2,Senior Database Administrator,...,0.0,0.0,INDIVIDUAL,0,57th week,0.0,0.0,444991.0,21400.0,0
2,27999917,11050,11050,11050.0,60 months,BAT446479,15.61,D,D1,Customer service representative,...,0.0,26.0,INDIVIDUAL,0,70th week,0.0,0.0,105737.0,16300.0,0
3,61514932,35000,35000,34700.0,60 months,BAT4664105,12.69,C,C2,ACCT OFFICER,...,0.0,0.0,INDIVIDUAL,0,22th week,0.0,0.0,287022.0,72400.0,0
4,59622821,6500,6500,6500.0,36 months,,6.89,A,A3,Paralegal,...,0.0,0.0,INDIVIDUAL,0,22th week,0.0,0.0,234278.0,26700.0,0


In [62]:
train_data.groupby('loan_status').mean()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,...,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,mths_since_last_major_derog,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
loan_status,,,,,,,,,,,,,,,,,,,,,
0,4.157725e+07,15206.679337,15201.751779,15183.324598,13.236660,75341.627145,18.762716,0.336317,0.646159,16.919046,...,1708.779919,0.404861,59.865701,6.362958,0.016808,11.938205,0.005649,221.169754,135516.179358,31746.694140
1,1.376923e+07,13306.413965,13265.955638,13159.017907,13.263356,74020.547055,16.122518,0.243739,0.851057,15.663562,...,1897.708430,0.362939,0.000000,0.000000,0.006183,8.052215,0.002964,117.686951,106015.967861,22450.231739


In [63]:

target_fields = ['loan_status', 'member_id']

features, targets = train_data.drop(target_fields, axis=1), train_data[target_fields]

test_features, test_targets = test_data.drop(target_fields, axis=1), test_data[target_fields]

In [64]:
train_features, train_targets = features[:-60*24], targets[:-60*24]
val_features, val_targets = features[-60*24:], targets[-60*24:]

In [65]:

%matplotlib inline

In [66]:

y, X = patsy.dmatrices('loan_status ~ member_id + loan_amnt + funded_amnt + funded_amnt_inv + annual_inc  + dti',
                  train_data, return_type="dataframe")
print(X.columns)

Index(['Intercept', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'annual_inc', 'dti'],
      dtype='object')


In [67]:
# flatten y into a 1-D array
y = np.ravel(y)

In [68]:
import scipy

In [69]:
from sklearn import linear_model

In [70]:
# instantiate a logistic regression model, and fit with X and y
model = linear_model.LogisticRegression()
model = model.fit(X, y)

# check the accuracy on the training set
model.score(X, y)

0.82023484865559282

In [71]:
# load dataset
testdta = test_data


In [72]:

test_y, test_X = patsy.dmatrices('loan_status ~ member_id + loan_amnt + funded_amnt + funded_amnt_inv + annual_inc + dti',
                  testdta, return_type="dataframe")
print(test_X.columns)

Index(['Intercept', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'annual_inc', 'dti'],
      dtype='object')


In [73]:
test_y = np.ravel(test_y)

In [74]:
model2 = linear_model.LogisticRegression()
model2.fit(X, y)

# check the accuracy on the training set
model.score(X, y)

0.82023484865559282

In [75]:
# predict class labels for the test set
predicted = model2.predict(test_X)
print(predicted)

[ 0.  0.  0. ...,  1.  0.  0.]


In [76]:
# generate class probabilities
probs = model2.predict_proba(test_X)
print(probs)

[[ 0.61251003  0.38748997]
 [ 0.87212085  0.12787915]
 [ 0.80505531  0.19494469]
 ..., 
 [ 0.46868176  0.53131824]
 [ 0.93935901  0.06064099]
 [ 0.50487948  0.49512052]]


In [77]:
from sklearn import metrics

In [78]:
# generate evaluation metrics
print(metrics.accuracy_score(test_y, predicted))
print(metrics.roc_auc_score(test_y, probs[:, 1]))

0.698524585084
0.498997952313


In [79]:
print(metrics.confusion_matrix(test_y, predicted))
print(metrics.classification_report(test_y, predicted))

[[237734  33473]
 [ 73536  10208]]
             precision    recall  f1-score   support

        0.0       0.76      0.88      0.82    271207
        1.0       0.23      0.12      0.16     83744

avg / total       0.64      0.70      0.66    354951



In [80]:
from sklearn import model_selection

from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression

In [81]:
model.predict_proba(np.array([1, 35023176, 3000, 3000, 3000,24000,3245]))

D:\Anaconda3\envs\dlnd-tf-lab\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 0.8746441,  0.1253559]])

In [82]:
model.predict_proba(test_X)

array([[ 0.61251003,  0.38748997],
       [ 0.87212085,  0.12787915],
       [ 0.80505531,  0.19494469],
       ..., 
       [ 0.46868176,  0.53131824],
       [ 0.93935901,  0.06064099],
       [ 0.50487948,  0.49512052]])

In [83]:
model.predict_proba(test_X)[1][1]

0.12787914725604546

In [84]:
model.predict_proba(test_X)[0][1]

0.38748997260621665

In [85]:
print(model.predict_proba(test_X).shape[0])

354951


In [86]:
probab = []

max_limit = model.predict_proba(test_X).shape[0] - 1 


for i in range(0,50):
   
    probab.append(model.predict_proba(test_X)[i][1])
    
probab = np.array(probab)


    

In [87]:
print(probab)

[ 0.38748997  0.12787915  0.19494469  0.04444037  0.03651016  0.17374025
  0.39576064  0.03806828  0.15900617  0.57925569  0.35836924  0.09110906
  0.0588843   0.45777374  0.02369564  0.03067014  0.31309311  0.49318647
  0.40716584  0.33847965  0.39502786  0.1382397   0.53051054  0.23191793
  0.12956439  0.08098481  0.04332998  0.32525899  0.04627947  0.02567282
  0.02292244  0.37202467  0.02414957  0.38467442  0.04173047  0.31479152
  0.34777009  0.0525887   0.10311096  0.38800247  0.53453324  0.31630786
  0.02569561  0.02704944  0.0660075   0.07627121  0.19207409  0.02756822
  0.04901615  0.09379747]


In [88]:
print(test_data.member_id[:50])

0     11937648
1     38983318
2     27999917
3     61514932
4     59622821
5     28822038
6     10718089
7     58114582
8     35023176
9      1268247
10    15559751
11    41234300
12    53958774
13     8074949
14    67601509
15    65560239
16    14767770
17     1726792
18     9899439
19    15120294
20    12019444
21    35242810
22     1057101
23    23024164
24    45430320
25    46396580
26    58832624
27    14990008
28    57738287
29    66382471
30    69172510
31    12096725
32    69625122
33    21137778
34    57619455
35    16260256
36    14318798
37    55256601
38    39394049
39    11039175
40     2917982
41    17792691
42    65459581
43    65534514
44    49505658
45    49844455
46    31147531
47    70089344
48    54544839
49    44477220
Name: member_id, dtype: int64
